In [21]:
import anatomist.api as anatomist

In [22]:
from soma import aims
import json
from soma.qt_gui.qt_backend import Qt
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np

In [23]:
a = anatomist.Anatomist()

In [24]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

result_file = "/neurospin/dico/agaudin/Runs/09_new_repo/Output/grid_searches/step2/schiz_stats_by_region.csv" 

result_ensemble_file = "/neurospin/dico/agaudin/Runs/09_new_repo/Output/grid_searches/step2/ensemble_models_by_region.csv"

In [25]:
with open(json_regions) as f:
    regions = json.load(f)

In [26]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [27]:
res = pd.read_csv(result_file)
res = res.rename(columns={"Unnamed: 0": "region_simple_name", "full_name": "region"})
res["region"].replace({"S.or.": "S.Or.", "S.or.-S.Olf.": "S.Or.-S.Olf.",}, inplace=True)
# res = res[~(res["region"] == "fronto-parietal_medial_face.")]
# res = res[~(res["region"] == "S.C.-S.Pe.C.")]
# res = res[~(res["region"] == "S.C.-S.Po.C.")]
res["region"].tolist()

['CINGULATE.',
 'S.F.inf.-BROCA-S.Pe.C.inf.',
 'Sc.Cal.-S.Li.',
 'S.T.s.',
 'S.T.s.br.',
 'S.C.-sylv.',
 'S.C.-S.Pe.C.',
 'S.C.-S.Po.C.',
 'S.Or.',
 'S.Or.-S.Olf.',
 'F.C.M.post.-S.p.C.',
 'S.F.median-S.F.pol.tr.-S.F.sup.',
 'S.F.inter.-S.F.sup.',
 'S.F.marginal-S.F.inf.ant.',
 'S.F.int.-S.R.',
 'S.T.i.-S.O.T.lat.',
 'OCCIPITAL',
 'F.I.P.',
 'S.s.P.-S.Pa.int.',
 'Lobule_parietal_sup.',
 'S.T.i.-S.T.s.-S.T.pol.',
 'F.Coll.-S.Rh.',
 'F.C.L.p.-subsc.-F.C.L.a.-INSULA.',
 'F.P.O.-S.Cu.-Sc.Cal.',
 'fronto-parietal_medial_face.']

In [28]:
res.loc[16, "best_test_auc"] = 0.5662589348635859
res[res["region"]=="OCCIPITAL"]

,region_simple_name,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final
16,occipital,OCCIPITAL,0.550496,0.566259,0.005175,0.004674,24.769073,13.494336,0.586651,0.597463,0.009773,0.008982,3.688913,3.349878


In [29]:
res = res[~(res["region"] == "fronto-parietal_medial_face.")]

In [30]:
def compute_logp(auc, std_auc):
    p = scipy.stats.ttest_ind_from_stats(mean1=auc, 
                              std1=std_auc, 
                              nobs1=5, 
                              mean2=0.5, 
                              std2=0, 
                              nobs2=2, 
                              alternative='greater',
                              equal_var=False)[1]
    return -np.log10(p)

In [31]:
# res["-log10_p"] =  res.apply(lambda x: compute_logp(x.AUC, x.std_auc), axis=1)
# res

In [32]:
def get_sulci(region):
    list_sulci = list(regions['brain'][f"{region}_right"].keys())
    list_sulci = ["S.F.int._right" if x=="paracingular._right" else x for x in list_sulci]
    return list_sulci

In [33]:
get_sulci("CINGULATE.")

['S.F.int._right', 'F.C.M.ant._right']

In [34]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region_simple_name,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final,sulcus
0,cingulate,CINGULATE.,0.532325,0.551791,0.006604,0.005809,12.103088,5.399155,0.585110,0.543461,0.008360,0.019361,1.355780,3.581346,"[S.F.int._right, F.C.M.ant._right]"
1,BROCA,S.F.inf.-BROCA-S.Pe.C.inf.,0.509573,0.515282,0.004978,0.003315,4.957421,1.527788,0.500891,0.539656,0.011979,0.011790,1.850623,0.325930,"[S.F.inf._right, S.F.inf.ant._right, F.C.L.r.a..."
2,pericalcarine,Sc.Cal.-S.Li.,0.525666,0.531585,0.004609,0.003554,12.056517,6.479759,0.557747,0.547025,0.013569,0.020130,1.399449,2.183796,"[F.Cal.ant.-Sc.Cal._right, S.O.p._right, S.Li...."
3,STs,S.T.s.,0.522412,0.553065,0.006333,0.004731,15.627258,3.394259,0.537043,0.572788,0.020517,0.011215,2.837745,1.138738,[S.T.s._right]
4,STs_br,S.T.s.br.,0.513011,0.540242,0.004988,0.003440,16.584515,2.240244,0.539354,0.524826,0.003447,0.012663,1.216549,3.774895,"[F.I.P.r.int.1_right, F.I.P.r.int.2_right, S.T..."


In [35]:
res = res.sort_values(by="test_auc", ascending=True)
res = res.explode("sulcus")
res.head()

,region_simple_name,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final,sulcus
9,Sor_olfactif,S.Or.-S.Olf.,0.490450,0.477443,0.005483,0.005369,1.974030e-05,0.019265,0.496074,0.453519,0.021415,0.014674,0.007432,0.245441,S.Or._right
9,Sor_olfactif,S.Or.-S.Olf.,0.490450,0.477443,0.005483,0.005369,1.974030e-05,0.019265,0.496074,0.453519,0.021415,0.014674,0.007432,0.245441,S.Olf._right
8,Sor,S.Or.,0.495816,0.461522,0.005502,0.003772,2.603684e-15,0.110689,0.458124,0.449381,0.015347,0.006541,0.000326,0.011556,S.Or._right
14,SFinternal,S.F.int.-S.R.,0.499814,0.505045,0.005653,0.005271,7.664323e-01,0.289810,0.511779,0.458502,0.022160,0.019588,0.022621,0.506404,S.F.int._right
14,SFinternal,S.F.int.-S.R.,0.499814,0.505045,0.005653,0.005271,7.664323e-01,0.289810,0.511779,0.458502,0.022160,0.019588,0.022621,0.506404,S.R.inf._right


In [36]:
for _, row in res.iterrows():
    print(row.sulcus)

S.Or._right
S.Olf._right
S.Or._right
S.F.int._right
S.R.inf._right
S.R.sup._right
F.I.P._right
S.Po.C.sup._right
S.Pa.sup._right
S.F.median._right
S.F.polaire.tr._right
S.F.sup._right
S.s.P._right
S.Pa.int._right
S.F.marginal._right
S.F.orbitaire._right
S.F.inf.ant._right
S.F.inf._right
S.F.inf.ant._right
F.C.L.r.ant._right
F.C.L.r.asc._right
F.C.L.r.diag._right
S.Pe.C.inf._right
S.C._right
S.C.sylvian._right
S.Po.C.sup._right
F.I.P.Po.C.inf._right
F.C.L.r.retroC.tr._right
F.I.P.r.int.1_right
F.I.P.r.int.2_right
S.T.s.ter.asc.ant._right
S.T.s.ter.asc.post._right
S.GSM._right
S.F.inter._right
S.F.sup._right
F.I.P._right
F.I.P.Po.C.inf._right
S.T.i.ant._right
S.T.i.post._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
S.C._right
S.C.sylvian._right
S.T.s._right
F.C.M.post._right
S.C.LPC._right
S.p.C._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.Li.ant._right
S.Li.post._right
S.C._right
S.C.sylvian._right
S.Pe.C.inf._right
S.Pe.C.inter._right
S.P

In [37]:
res[res.sulcus=="S.F.orbitaire._right"]

,region_simple_name,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final,sulcus
13,SFmarginal,S.F.marginal-S.F.inf.ant.,0.508136,0.509374,0.004883,0.004235,1.813174,1.296782,0.499607,0.528934,0.009392,0.014437,1.238216,0.287627,S.F.orbitaire._right


In [38]:
g = aims.read(Rspam_model)
ag = a.toAObject(g)



for vertex in g.vertices():
    vertex['AUC'] = 0.

for _, row in res.iterrows():
    for vertex in g.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['best_test_auc']):
                vertex['AUC'] = row.best_test_auc

ag.setColorMode(ag.PropertyMap)
ag.setColorProperty('AUC')
ag.notifyObservers()

w = a.createWindow('3D')
w.addObjects(ag)


In [39]:
res = res.sort_values(by="-log_p_value_best", ascending=True)

In [40]:
g2 = aims.read(Rspam_model)
ag2 = a.toAObject(g2)

for vertex in g2.vertices():
    vertex['-log10p'] = 0.

for _, row in res.iterrows():
    for vertex in g2.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['-log_p_value_best']):
                vertex['-log10p'] = row['-log_p_value_best']

ag2.setColorMode(ag2.PropertyMap)
ag2.setColorProperty('-log10p')
ag2.notifyObservers()

w2 = a.createWindow('3D')
w2.addObjects(ag2)


: 